In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from bs4 import BeautifulSoup

fields = ['t83','t87','t91','t95','t99','t103','t107','t109','t115','t119','t123','t129','t134','t158','t164','t170']
keys = ['RNEC Number',
        'EudraCT Number',
        'Protocol Number',
        'Type of Study',
        "Sponsor's Name",
        'Title of the trial',
        'Recruitment Status',
        'Date of Global End',
        'Infarmed Authorization Date',
        'CEC Opinion Date',
        'Non-technical title',
        'Paediatric Investigation Plan',
        'Medical condition(s)',
        'Therapeutic Confirmatory (Phase II)',
        'Therapeutic Confirmatory (Phase III)',
        'Therapeutic Confirmatory (Phase IV)',
        'Age_Range',
        'Gender'
        ]


data = []
df = pd.DataFrame(columns=keys)


driver = webdriver.Chrome()
driver.get("https://www.rnec.pt/pesquisa-de-estudos-clinicos")
study_num = (0, 1)



# Initialization -  search button click and Html Refresh (clicks page1)
search_button = driver.find_element(By.ID, '_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt73').click()
driver.implicitly_wait(15)
page_1_update = driver.find_element(By.XPATH, '//*[@id="_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt79_paginator_bottom"]/span[3]/span[1]').click()

for page in range (1,298):
    page_html = driver.page_source
    soup = BeautifulSoup(page_html, 'html.parser')
    for study in study_num:
        study_data = []
        for field in fields:
            try:
                element = soup.find(id=f'_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt79:{study}:j_id{field}')['value']
                         
            except:
                element = None
            study_data.append(element)
        try:
            age_range= soup.find('label', {'id': f'_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt79:{study}:j_idt173'}).parent.find_next_sibling().text.strip()
            gender = soup.find('label', {'id': f'_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt79:{study}:j_idt181'}).parent.find_next_sibling().text.strip()
        except:
            age_range = None
            gender = None
        study_data.append(age_range)
        study_data.append(gender)
        data.append(study_data)
    
    study_num = tuple([x + 2 for x in study_num])
    next_page = driver.find_element(By.XPATH, '//*[@id="_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt79_paginator_bottom"]/span[4]/span').click()
    time.sleep(5)
    
    
dict = [{k:v for k,v in zip(keys, d)} for d in data]
df = df.append(dict, ignore_index=True)
df.to_csv('RNEC.csv', index=False)

C:\Users\Diogo\AppData\Local\Temp\ipykernel_31784\3642016529.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict, ignore_index=True)


In [3]:
df.isna().sum()

RNEC Number                               0
EudraCT Number                            0
Protocol Number                           0
Type of Study                             0
Sponsor's Name                            0
Title of the trial                        0
Recruitment Status                       32
Date of Global End                      594
Infarmed Authorization Date               0
CEC Opinion Date                          2
Non-technical title                       2
Paediatric Investigation Plan             0
Medical condition(s)                      0
Therapeutic Confirmatory (Phase II)     432
Therapeutic Confirmatory (Phase III)    164
Therapeutic Confirmatory (Phase IV)     562
Age_Range                                 0
Gender                                    0
dtype: int64